In [2]:
import re
import math
import nltk
import numpy as np
import pandas as pd 
from scipy import sparse
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

Doc1 = "So far in this course, we have studied association rule mining and we learned about frequent itemsets"
Doc2 = "During the past couple of weeks, we started learning about Text Mining"
Doc3 = "Data Mining and Text Mining are fun topics!"
Doc4 = "Holidays are fun! I can’t wait till my next holiday…"
Docs = [Doc1,Doc2,Doc3,Doc4]

stopwords = stopwords.words('english')

def preprocessCorpus(corpus):
    Do1, Do2, Dox = [], [], []

    for Doc in Docs:
        Do1.append(Doc.casefold())
        
    for Doc in Do1:
        curr = ""
        for word in  re.split("\W+",Doc):
            if word not in stopwords: 
                curr = curr + word +" "
        curr = curr.strip()
        Do2.append(curr)

    for Doc in  Do2:
        curr = ""
        for word in Doc.split():  
            curr = curr + PorterStemmer().stem(word) +" "
        curr = curr.strip()
        Dox.append(curr)
    return Dox

Dox = preprocessCorpus(Docs)
print(Dox)

vectorizer = CountVectorizer()
V = vectorizer.fit_transform(Dox).toarray()

['far cours studi associ rule mine learn frequent itemset', 'past coupl week start learn text mine', 'data mine text mine fun topic', 'holiday fun wait till next holiday']


In [3]:
def getTF_IDF(corpus):
    TF, IDF, TF_IDF, N = [], [], [], []
    for i in range(4):
        tf = V[i] #/ len(Dox[i].split()) || 1 + log(V[i] / len(Dox[i].split()))
        TF.append(tf)

    index = 0
    Sum = [0]*21
    for i in V.T:    
        if 2 not in i:
            Sum[index] = sum(i)
        elif 2 in i:
            Z = sum(i)
            Sum[index] = Z - 1
        index = index+1

    for i in range(len(Sum)):
        idf = math.log10(len(Dox))/Sum[i]
        IDF.append(idf)

    for i in range(len(V)):
        TF_idf = TF[i] * IDF
        TF_IDF.append(TF_idf)
        
    #for i in range(4):
    #    normalized = (TF_IDF[i]-min(TF_IDF[i]))/(max(TF_IDF[i])-min(TF_IDF[i]))
    #    N.append(normalized)
    Sp = sparse.csr_matrix(TF_IDF)#to.array()
    return Sp

IDF_vec = getTF_IDF(Dox).toarray()
print(IDF_vec)

[[0.60205999 0.         0.60205999 0.         0.60205999 0.60205999
  0.         0.         0.60205999 0.30103    0.20068666 0.
  0.         0.60205999 0.         0.60205999 0.         0.
  0.         0.         0.        ]
 [0.         0.60205999 0.         0.         0.         0.
  0.         0.         0.         0.30103    0.20068666 0.
  0.60205999 0.         0.60205999 0.         0.30103    0.
  0.         0.         0.60205999]
 [0.         0.         0.         0.60205999 0.         0.
  0.30103    0.         0.         0.         0.40137333 0.
  0.         0.         0.         0.         0.30103    0.
  0.60205999 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.30103    1.20411998 0.         0.         0.         0.60205999
  0.         0.         0.         0.         0.         0.60205999
  0.         0.60205999 0.        ]]


In [27]:
def Sim1(v1, v2):
    result = np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))
    return result

In [28]:
Q = ['Text mining'][0].split()
X = []
curr = ""
for word in Q:  
    curr = curr + PorterStemmer().stem(word) +" "
curr = curr.strip()
X.append(curr)
v1 = vectorizer.transform(X).toarray()
index = 0
Sum = [0]*21
for i in V.T:    
    if 2 not in i:
        Sum[index] = sum(i)
    elif 2 in i:
        Z = sum(i)
        Sum[index] = Z - 1
    index = index+1
IDFv1 = []
for i in v1:
    index = 0
    for x in i:
        x = x * math.log10(len(Dox))/Sum[index]
        IDFv1.append(x)
    index += 1
v1 = np.array(IDFv1).reshape(-1,1).T

Cos = []
for Doc in IDF_vec:
    Cos.append(Sim1(v1,Doc))
Cos.sort(reverse=True)
print(Cos)

[array([0.48076197]), array([0.27441065]), array([0.08687445]), array([0.])]
